# Gaussova eliminacija

## Općenito

Sustav $Ax=b$ se rješava u tri koraka (__bez pivotiranja__)

1. $A=LU$ (LU rastav, $O(\frac{2}{3}n^3)$ operacija)
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija)
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija)

Ukoliko pivotiramo, tada je

1. $PA=LU$ 
2. $Ly=P^T b$
3. $Ux=y$ 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6x6 Array{Float64,2}:
 0.572742  0.345477  0.41618   0.1079    0.0915292  0.711516  
 0.918662  0.406217  0.621473  0.322772  0.382369   0.00569496
 0.466936  0.780852  0.46122   0.251193  0.994715   0.805163  
 0.911242  0.9977    0.145054  0.624245  0.872087   0.94439   
 0.388126  0.033171  0.538476  0.153353  0.648229   0.306619  
 0.9793    0.395671  0.992422  0.609758  0.211017   0.306964  

In [3]:
L,U=mylu(A)

(
6x6 Array{Float64,2}:
 1.0        0.0        0.0       0.0         0.0     0.0
 1.60397    1.0        0.0       0.0         0.0     0.0
 0.815264  -3.37483    1.0       0.0         0.0     0.0
 1.59102   -3.02899   19.5732    1.0         0.0     0.0
 0.677662   1.35849   -9.50975  -0.511892    1.0     0.0
 1.70984    1.31857  -10.1814   -0.577586  -10.4546  1.0,

6x6 Array{Float64,2}:
 0.572742   0.345477   0.41618      0.1079      0.0915292   0.711516
 0.0       -0.147918  -0.0460677    0.149703    0.235559   -1.13556 
 0.0        0.0       -0.0335476    0.66845     1.71507    -3.60723 
 0.0        0.0        0.0        -12.1777    -32.1293     66.9777  
 0.0        0.0        0.0          0.0         0.129292    1.34863 
 0.0        0.0        0.0          0.0         0.0        16.6457  )

In [4]:
L*U-A

6x6 Array{Float64,2}:
 0.0  0.0  0.0   0.0           0.0           0.0        
 0.0  0.0  0.0   0.0           0.0           0.0        
 0.0  0.0  0.0   0.0           0.0           0.0        
 0.0  0.0  0.0  -1.55431e-15  -1.77636e-15  -3.77476e-15
 0.0  0.0  0.0   0.0           1.11022e-15   3.33067e-15
 0.0  0.0  0.0  -4.44089e-16   1.77636e-15   8.88178e-16

## Trokutasti sustavi

In [5]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [6]:
b=rand(6)

6-element Array{Float64,1}:
 0.0243455 
 0.930382  
 0.534567  
 0.00546513
 0.310271  
 0.0227217 

In [7]:
x=A\b

6-element Array{Float64,1}:
  0.905914
  0.916703
  0.28446 
 -2.17203 
  0.669806
 -1.06328 

In [8]:
y=myL(L,b)

6-element Array{Float64,1}:
   0.0243455
   0.891332 
   3.52282  
 -66.2862   
  -1.34736  
 -17.699    

In [9]:
x1=myU(U,y)

6-element Array{Float64,1}:
  0.905914
  0.916703
  0.28446 
 -2.17203 
  0.669806
 -1.06328 

In [10]:
x-x1

6-element Array{Float64,1}:
 -6.77236e-15
 -8.88178e-15
  1.95954e-14
 -1.24345e-14
  4.21885e-15
 -2.22045e-16

## Brzina

Program `mylu()` je jako spor. Između ostalog, alocira nepotrebno tri matrice.

Program se može reformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema, jer su svi elementi jednaki 1.

In [11]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]*A[k,rho]
    end
    A
end

mylu1 (generic function with 1 method)

In [12]:
mylu1(A)

6x6 Array{Float64,2}:
 0.572742   0.345477    0.41618      0.1079      0.0915292   0.711516
 1.60397   -0.147918   -0.0460677    0.149703    0.235559   -1.13556 
 0.815264  -3.37483    -0.0335476    0.66845     1.71507    -3.60723 
 1.59102   -3.02899    19.5732     -12.1777    -32.1293     66.9777  
 0.677662   1.35849    -9.50975     -0.511892    0.129292    1.34863 
 1.70984    1.31857   -10.1814      -0.577586  -10.4546     16.6457  

In [13]:
L,U

(
6x6 Array{Float64,2}:
 1.0        0.0        0.0       0.0         0.0     0.0
 1.60397    1.0        0.0       0.0         0.0     0.0
 0.815264  -3.37483    1.0       0.0         0.0     0.0
 1.59102   -3.02899   19.5732    1.0         0.0     0.0
 0.677662   1.35849   -9.50975  -0.511892    1.0     0.0
 1.70984    1.31857  -10.1814   -0.577586  -10.4546  1.0,

6x6 Array{Float64,2}:
 0.572742   0.345477   0.41618      0.1079      0.0915292   0.711516
 0.0       -0.147918  -0.0460677    0.149703    0.235559   -1.13556 
 0.0        0.0       -0.0335476    0.66845     1.71507    -3.60723 
 0.0        0.0        0.0        -12.1777    -32.1293     66.9777  
 0.0        0.0        0.0          0.0         0.129292    1.34863 
 0.0        0.0        0.0          0.0         0.0        16.6457  )

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [14]:
n=512
A=rand(n,n)

512x512 Array{Float64,2}:
 0.879008    0.0765234  0.619828    …  0.306852  0.419349   0.0318452
 0.978776    0.342315   0.584766       0.470737  0.817043   0.697399 
 0.00211954  0.600403   0.723049       0.913511  0.540738   0.383263 
 0.0513583   0.548405   0.722061       0.72406   0.36884    0.583743 
 0.673156    0.630861   0.45636        0.654216  0.293304   0.0456388
 0.434668    0.936868   0.866398    …  0.860935  0.177937   0.131977 
 0.0486402   0.895425   0.773846       0.051719  0.793414   0.0656182
 0.335845    0.345502   0.21187        0.59085   0.375575   0.531287 
 0.233454    0.708725   0.817826       0.529515  0.484588   0.783603 
 0.635393    0.829388   0.460663       0.131837  0.895702   0.543087 
 0.716301    0.633068   0.526266    …  0.368188  0.241099   0.994956 
 0.0598042   0.119478   0.943496       0.74961   0.455477   0.750463 
 0.191756    0.811144   0.0686469      0.23966   0.712608   0.716227 
 ⋮                                  ⋱            ⋮              

In [15]:
@time lu(A);

In [16]:
@time mylu1(A);

0.153377 seconds (177.58 k allocations: 14.420 MB, 3.03% gc time)
  

### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [17]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [18]:
# Probajte k,l=32,16 i k,l=64,8
k,l=4,4
Ab=[rand(k,k) for i=1:l, j=1:l];

In [19]:
A0=mylu2(Ab)

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.678858  0.662695   0.295691  0.720758
 0.075622  0.855589   0.746546  0.449273
 0.901516  0.301884   0.503178  0.456996
 0.45916   0.0633566  0.948613  0.423762                              …  4x4 Array{Float64,2}:
 0.518495  0.718859  0.662116   0.30917  
 0.4364    0.769264  0.795726   0.0751198
 0.844604  0.108336  0.525455   0.727627 
 0.925806  0.229162  0.0192927  0.147921             
 4x4 Array{Float64,2}:
 -1.35546    0.737376    2.21878   -0.716719
  0.671226  -0.0250903  -0.900465   0.853136
 -0.225808   0.784549    1.55605   -0.999857
  1.37584    0.205724   -1.26639    0.916994                 4x4 Array{Float64,2}:
 -0.283142    0.544117  -0.212943  -1.06509 
  0.0823935   0.207579   0.370494   0.888356
 -0.115731   -0.195113  -0.56117   -0.620051
 -0.564426   -0.287248  -0.331463   1.08581 
 4x4 Array{Float64,2}:
  0.931489    0.0660938   -0.960304   1.18999  
  0.960611    0.00858041  -0.75833    0.441672 
 -0.0297255   0.07654

0.413389 seconds (17.26 k allocations: 1.004 GB, 8.47% gc time)


In [20]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [21]:
Res=L*U-Ab

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                                  …  4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                            
 4x4 Array{Float64,2}:
 -2.22045e-16   1.66533e-16  -1.11022e-16  -5.55112e-17
  2.22045e-16   0.0           0.0           0.0        
  0.0          -2.22045e-16  -2.22045e-16  -5.55112e-17
  1.11022e-16   1.11022e-16   0.0          -2.22045e-16     4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                         

In [22]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [23]:
norm(unblock(Res))

8.207746519412836e-15

Sada probajmo veću dimenziju

In [24]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [25]:
@time mylu2(Ab);

Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Porgeska_unatrag_i stabilni_algoritmi](NA04_Pogeska_unatrag_i stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za danai problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$
za neki $\hat x=x+\delta x$.

Želimo ocijeniti 
$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [G. Golub and C. F. Van Loan, Matrix Computations, str. 87, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\dot x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$
odnosno
$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $x(0)$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$
odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,
$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$ imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je _uvjetovanost_ (_kondicija_)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [R. Scitovski, Numerička matematika, str. 42][RS04]:




[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, osijek, 2004."

In [26]:
A= [0.234 0.458; 0.383 0.750]

2x2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [27]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [28]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [29]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

0.048136 seconds (8.79 k allocations: 26.526 MB, 7.00% gc time)


In [30]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605,0.0002096449170953002,0.6020311134825742)

In [31]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [32]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605,0.0010134105230118603,0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [G. Golub and C. F. Van Loan, Matrix Computations, str. 122, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(n^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1) slijedi 
da za relativnu pogreška izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

In [33]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.47599 
 0.559261
 0.328928
 0.897325
 0.425351
 0.478643
 0.258632
 0.834128
 0.183083
 0.387216

In [34]:
A=Array(Float64,n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

10x10 Array{Float64,2}:
 1.0         1.0         1.0          …  1.0       1.0          1.0        
 0.47599     0.559261    0.328928        0.834128  0.183083     0.387216   
 0.226567    0.312773    0.108194        0.695769  0.0335195    0.149936   
 0.107843    0.174922    0.035588        0.580361  0.00613685   0.0580578  
 0.0513324   0.0978272   0.0117059       0.484095  0.00112355   0.0224809  
 0.0244337   0.0547109   0.0038504    …  0.403797  0.000205704  0.00870498 
 0.0116302   0.0305977   0.00126651      0.336819  3.76609e-5   0.00337071 
 0.00553587  0.0171121   0.00041659      0.28095   6.89508e-6   0.00130519 
 0.00263502  0.00957015  0.000137028     0.234348  1.26237e-6   0.000505392
 0.00125424  0.00535222  4.50725e-5      0.195476  2.3112e-7    0.000195696

In [35]:
x=rand(n)

10-element Array{Float64,1}:
 0.553354 
 0.847866 
 0.0578554
 0.32206  
 0.758364 
 0.260547 
 0.0155135
 0.578822 
 0.148934 
 0.517928 

In [36]:
b=A*x

10-element Array{Float64,1}:
 4.06124 
 2.20751 
 1.33945 
 0.896848
 0.651413
 0.502883
 0.404916
 0.335168
 0.282392
 0.240646

In [37]:
cond(A)

5.3808151517104004e10

In [38]:
x1=A\b

10-element Array{Float64,1}:
 0.553354 
 0.847866 
 0.0578554
 0.32206  
 0.758364 
 0.260547 
 0.0155135
 0.578822 
 0.148934 
 0.517928 

In [39]:
norm(x1-x)/norm(x)

1.6509389310821128e-7